In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install catboost
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

import tqdm
import gc
import math

import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime, timedelta
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

# deeplearning
import tensorflow as tf
from tensorflow.keras import layers  
# from sklearn import tree
# from sklearn.tree import export_graphviz

# machine learning
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

# regression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier

# svm
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 
from catboost import Pool,CatBoostClassifier

import os
import pandas as pd
import numpy as np

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Input, Dense, Activation, Reshape, Concatenate
from keras.layers.embeddings import Embedding
import matplotlib.pyplot as plt
from keras.models import Model as KerasModel
import bisect

In [ ]:


path_test = 'https://raw.githubusercontent.com/genie-lee-beep/Jieun-Lee/main/AI%20Competition/jobcare%20recommendation/test.csv'

path_train= 'https://raw.githubusercontent.com/genie-lee-beep/Jieun-Lee/main/AI%20Competition/jobcare%20recommendation/train.csv'

path_sub = 'https://raw.githubusercontent.com/genie-lee-beep/Jieun-Lee/main/AI%20Competition/jobcare%20recommendation/sample_submission.csv'

ori_train = pd.read_csv(path_train)
ori_test = pd.read_csv(path_test)
sample_submission = pd.read_csv(path_sub)

train_size = len(ori_train)
test_size = len(ori_test)

display(ori_train,ori_test)



,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,...,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target
0,0,True,True,True,False,False,False,1,4,3,...,2,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1
1,1,False,False,False,True,True,False,1,3,4,...,2,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0
2,2,False,False,False,True,False,False,2,0,3,...,1,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0
3,3,False,False,False,True,False,False,2,0,2,...,2,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0
4,4,True,True,True,False,False,False,1,3,4,...,2,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,501946,False,False,False,True,False,False,1,1,2,...,2,354,147,1,5,65,503156,285850,2020-03-13 12:55:52,1
501947,501947,True,True,False,True,False,False,1,6,2,...,2,163,120,1,4,142,676255,456996,2020-01-20 11:51:51,1
501948,501948,True,True,True,True,False,False,1,7,4,...,2,438,147,2,7,65,484528,293258,2020-08-05 17:27:24,1
501949,501949,True,False,False,True,False,False,1,1,2,...,2,660,147,3,4,259,456330,273797,2020-06-15 09:23:21,1


,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,...,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt
0,0,True,False,False,True,True,True,1,1,2,...,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18
1,1,False,False,False,True,False,False,2,0,2,...,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53
2,2,True,False,False,True,True,True,2,3,2,...,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41
3,3,True,False,False,True,True,True,1,2,2,...,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55
4,4,True,False,False,True,False,False,1,6,4,...,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,46399,True,True,True,False,False,False,2,0,4,...,1,2,759,147,1,5,91,425040,726084,2020-12-17 01:06:26
46400,46400,True,False,False,True,False,False,2,0,4,...,1,2,759,147,1,4,91,290061,156948,2020-12-29 21:57:25
46401,46401,True,True,False,True,True,True,2,0,3,...,1,2,759,147,1,4,288,307951,175069,2020-12-10 19:09:09
46402,46402,True,True,False,True,True,True,1,3,3,...,3,2,759,147,1,5,288,308354,174849,2020-12-07 14:04:34


In [ ]:
ori_train.columns

Index(['id', 'd_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_rn', 'contents_rn', 'contents_open_dt',
       'target'],
      dtype='object')

In [ ]:
ori_test.columns

Index(['id', 'd_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_rn', 'contents_rn', 'contents_open_dt'],
      dtype='object')

In [ ]:
set(ori_train) - set(ori_test)

{'target'}

In [ ]:
# 콘텐츠 번호 범주형으로 묶을 수 있는지 확인 > 불가능
ori_train['contents_rn'].unique

<bound method Series.unique of 0         354805
1         346213
2         206408
3         572323
4         573899
           ...  
501946    285850
501947    456996
501948    293258
501949    273797
501950    176650
Name: contents_rn, Length: 501951, dtype: int64>

# [전처리함수]

Q) cols로 넣어주면 되지 왜 col에 담아주는 반복문을 만들었을까?

In [ ]:
# def check_diff(train,test):
#     '''
#     column별 test에만 있는 values들을 확인한다
#     '''
#     cols = test.columns
    
#     for col in cols:
#         print(col)  # test에 있는 전체 col
#         print(set(test[col]) - set(train[col]))  # test에만 있는 col
#         print(set(test[col]) == set(train[col]))  # train와 test에 공통으로 있는 col

In [ ]:
# 그냥 메모

def do_onehot_v2(train,test,cols):
  train_dummy_df = pd.get_dummies(train[col])
  test_dummy_df = pd.get_dummies(test[col])

  print(train_dummy_df)
  print(test_dummy_df)

'''
이 자리(def와 같은 라인)에서 print를 하면 train_dummy_df가 정의되지 않다고 하는 이유: 
train_dummy_df는 do_onehot_v2 함수 안에서 정의된 지역변수이기 때문이다. 
따라서 위와 같이 print(train_dummy_df)는 do_onehot_v2 안으로 적어 주어야 한다. 

print(train_dummy_df) 
print(test_dummy_df)  
'''

'\n이 자리(def와 같은 라인)에서 print를 하면 train_dummy_df가 정의되지 않다고 하는 이유: \ntrain_dummy_df는 do_onehot_v2 함수 안에서 정의된 지역변수이기 때문이다. \n따라서 위와 같이 print(train_dummy_df)는 do_onehot_v2 안으로 적어 주어야 한다. \n\nprint(train_dummy_df) \nprint(test_dummy_df)  \n'

# One hot encoing

In [ ]:
pd.get_dummies(ori_train)

NameError: ignored

# Scalling

In [ ]:
train[v].describe()

In [ ]:
def do_scaling(train,test,cols,type):
    from sklearn import preprocessing
    '''
    scaling by 'type'
    '''

    train_copy = train.copy()
    test_copy = test.copy()

    if type=="minmax":
        scaler = preprocessing.MinMaxScaler()
    elif type=="standard":
        scaler = preprocessing.StandardScaler()
    elif type=="robust":
        scaler = preprocessing.RobustScaler()
    elif type=="normalize":
        scaler = preprocessing.Normalizer()

    scaler_fit = scaler.fit(train_copy[cols])
    
    # [train]
    result=scaler_fit.transform(train_copy[cols])
    for j,col_name in enumerate(cols):
        train_copy[col_name] = result[:,j]

    # [test]
    result_te=scaler_fit.transform(test_copy[cols])
    for j,col_name in enumerate(cols):
        test_copy[col_name] = result_te[:,j]

    return train_copy,test_copy

# Label Encoding

In [ ]:
def do_label_encoding(train,test,cols):
    '''
    label encoding cols in train and transform by train
    '''
    from sklearn.preprocessing import LabelEncoder
    train_copy = train.copy()
    test_copy = test.copy()
    
    for col in cols:
        le=LabelEncoder()
        le.fit(train_copy[col])
        
        train_copy[col] = le.transform(train_copy[col])
        test_copy[col] = le.transform(test_copy[col])


    return (train_copy,test_copy)

def time_labeling(df,col,target_time):
    from datetime import datetime
    '''
    col : str type time column( ex : "2020-07-08 20:00:10", type:str)
    targer_time : target time to convert (ex:["year","month","day","weekday","hour"])
    to onehot train, test time col
    '''

    copied_df = df.copy()
    df_size = len(copied_df)

    time_df={target:[] for target in target_time}
    for i in range(df_size):
        dt = datetime.strptime(copied_df[col][i],"%Y-%m-%d %H:%M:%S")
        if "year" in time_df:
            time_df["year"].append(dt.year)
        if "month" in time_df:
            time_df["month"].append(dt.month)
        if "day" in time_df:
            time_df["day"].append(dt.day)
        if "weekday" in time_df:
            time_df["weekday"].append(dt.weekday())
        if "hour" in time_df:
            time_df["hour"].append(dt.hour)
    time_df=pd.DataFrame(time_df)

    copied_df = copied_df.drop(columns=[col],axis=1)
    copied_df = pd.concat([copied_df,time_df],axis=1)
    
    return copied_df

# [검증함수]

# [시각화 함수]

# [전처리, 3이 best]

# 1. D,H,L 컬럼 삭제

# 2. 일치여부만 컬럼으로 (다같아서 그냥 삭제)

# 3. 속성 세분화 및 일치여부 컬럼

a : 각컬럼,일치  
a_1 : 스캐일링  
b : 스캐일링  
c : 컬럼,일치  
d : 세분화일치  
e : 각컬럼, 일치  
f : 각컬럼  
g : 각컬럼  
h : 세분화일치  
i : 각컬럼  
j : 각컬럼  
k : 각컬럼  
l : 각컬럼(도메인 너무많음)  
m : 각컬럼  

# 4
(1) person, content 추가

# 5. person, content추가, 일치여부컬럼만 남기기


# 6 원핫코딩 x

# 7 도메인이 하나거나 비율이 낮은 컬럼 삭제

# [모델 학습]